In [65]:
#imports
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Read CSV file
df = pd.read_csv(os.environ['TRAIN_LOCATION'])

#rename columns
df.rename(columns={'Working Professional or Student': 'Working Student', 'Have you ever had suicidal thoughts ?': 'Suicidal Thoughts', 'Family History of Mental Illness':'Family Mental Illness' }, inplace=True)

# Convert all column names to snake_case
df.columns = (
    df.columns
    .str.strip()                              # remove leading/trailing spaces
    .str.replace(' ', '_')                    # replace spaces with underscores
    .str.replace('[^A-Za-z0-9_]+', '', regex=True)  # remove special characters
    .str.lower()            
                      # convert to lowercase (optional)
)

# View result
df.head()

,id,name,gender,age,city,working_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,suicidal_thoughts,workstudy_hours,financial_stress,family_mental_illness,depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [66]:
#Check for missing and duplicates
print(df.duplicated().sum()) # no duplicates
df.isna().sum() #We have alot of missing data that we will deal with in different ways

0


id                            0
name                          0
gender                        0
age                           0
city                          0
working_student               0
profession                36630
academic_pressure        112803
work_pressure             27918
cgpa                     112802
study_satisfaction       112803
job_satisfaction          27910
sleep_duration                0
dietary_habits                4
degree                        2
suicidal_thoughts             0
workstudy_hours               0
financial_stress              4
family_mental_illness         0
depression                    0
dtype: int64

In [67]:
# Pre processing

#Convert yes/no to binary
df = df.replace({'Yes': 1, 'No': 0}).infer_objects(copy=False)
#Make working_student binary, if they are working it will be 1, student 0 
df['working_student'] = df['working_student'].replace({'Working Professional':1, 'Student': 0})
#Male 1 Female 0 
df['gender'] = df['gender'].replace({'Male': 1, 'Female': 0})
#Dealing with Null values
#For profession if they are a student their profession is "Student"
# df.loc[df['working_student'] == 0, 'profession'] = 'Student'
display(df.sample(10))
print(df.isna().sum())
df_counts = df['sleep_duration'].value_counts().rename_axis('sleep_duration').reset_index(name='count')


C:\Users\nikan\AppData\Local\Temp\ipykernel_8448\898617091.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Yes': 1, 'No': 0}).infer_objects(copy=False)
C:\Users\nikan\AppData\Local\Temp\ipykernel_8448\898617091.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['working_student'] = df['working_student'].replace({'Working Professional':1, 'Student': 0})
C:\Users\nikan\AppData\Local\Temp\ipykernel_8448\898617091.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a fu

,id,name,gender,age,city,working_student,profession,academic_pressure,work_pressure,cgpa,study_satisfaction,job_satisfaction,sleep_duration,dietary_habits,degree,suicidal_thoughts,workstudy_hours,financial_stress,family_mental_illness,depression
16355,16355,Rajat,1,54.0,Agra,1,Content Writer,NaN,5.0,NaN,NaN,3.0,More than 8 hours,Moderate,BCA,1,9.0,3.0,1,0
95702,95702,Prachi,0,31.0,Visakhapatnam,0,NaN,2.0,NaN,5.60,2.0,NaN,Less than 5 hours,Moderate,B.Ed,0,4.0,2.0,1,0
67481,67481,Shreya,0,18.0,Agra,1,NaN,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Unhealthy,Class 12,1,7.0,5.0,1,1
78220,78220,Ishwar,1,19.0,Agra,1,NaN,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Unhealthy,Class 12,1,9.0,1.0,0,1
1911,1911,Nikhil,1,26.0,Bhopal,0,NaN,4.0,NaN,8.04,3.0,NaN,5-6 hours,Moderate,LLM,1,6.0,5.0,1,1
73801,73801,Yuvraj,1,60.0,Meerut,1,NaN,NaN,3.0,NaN,NaN,1.0,Less than 5 hours,Healthy,Class 12,1,3.0,3.0,0,0
114789,114789,Bhavesh,1,50.0,Agra,1,Content Writer,NaN,5.0,NaN,NaN,3.0,More than 8 hours,Healthy,MA,1,11.0,5.0,0,0
57915,57915,Riya,0,20.0,Vasai-Virar,0,NaN,1.0,NaN,6.37,1.0,NaN,More than 8 hours,Healthy,Class 12,0,6.0,5.0,0,0
68318,68318,Rupak,1,23.0,Hyderabad,0,NaN,3.0,NaN,8.73,5.0,NaN,More than 8 hours,Moderate,B.Arch,0,3.0,1.0,1,0
75323,75323,Ivaan,1,34.0,Agra,0,NaN,4.0,NaN,8.70,5.0,NaN,More than 8 hours,Moderate,M.Ed,0,2.0,1.0,0,0


id                            0
name                          0
gender                        0
age                           0
city                          0
working_student               0
profession                36630
academic_pressure        112803
work_pressure             27918
cgpa                     112802
study_satisfaction       112803
job_satisfaction          27910
sleep_duration                0
dietary_habits                4
degree                        2
suicidal_thoughts             0
workstudy_hours               0
financial_stress              4
family_mental_illness         0
depression                    0
dtype: int64


In [ ]:
from OurModels.LinearRegression import LinearRegression

# Create an instance
model = LinearRegression(df)

# Call the instance method
model.dede()  # prints df.head()


AttributeError: 'DataFrame' object has no attribute 'df'